### 1. Import required libraries

In [1]:
import os
import numpy as np

### 2. Reproducibility

In [2]:
# Thumb rule: Always set the seed
seed =1234
np.random.seed(seed)

### 3. Compute IOU
Suppose you have two tensors, one representing ground truth and the other one representing predictions. Each tensor
is having n number of boxes and we need to evaluate IOU for each pair. This implementation shows how to do it without creating an explicit for-loop. This is a vectorized version and hence much faster and scalable 

In [3]:
def compute_ious(boxesA, boxesB):
    """
    This function is used to compute IOU between pairs of 
    bouding boxes. 
    BEWARE: I am using the (xmin, ymin, xmax, ymax) notation
    in this implementation. If you are using some other format,
    please change the indices accordingly in the implementation.
    
    Args:
        boxesA: N-d(nxn) array containing boxes(e.g. Ground truth boxes)
        boxesB: N-d(nxn) array containing boxes(e.g. Predictions)
        
    Returns:
        IOU: N-D(nxn) array representing IOU for each pair
    """
    
    # sanity check
    if len(boxesA)!=len(boxesB):
        raise ValueError("Both arrays should contain same number of boxes")
        return
    
    # 1. Split the boxes into coordinates. 
    xminA, yminA, xmaxA, ymaxA = np.split(boxesA, 4, axis=1)
    xminB, yminB, xmaxB, ymaxB = np.split(boxesB, 4, axis=1)
    
    # 2. Compute Area for the boxes in both arrays
    areaA = (boxesA[:,2] - boxesA[:,0]) * (boxesA[:,3]-boxesA[:,1])
    areaB = (boxesB[:,2] - boxesB[:,0]) * (boxesB[:,3]-boxesB[:,1])
    
    # 3. Find min max y-coordinate for each pair
    # We are taking transpose because we need pairwaise min max
    all_pairs_min_ymax = np.minimum(ymaxA, np.transpose(ymaxB))
    all_pairs_max_ymin = np.maximum(yminA, np.transpose(yminB))
    
    # 4. Computer intersection in height
    intersect_heights = np.maximum(np.zeros(all_pairs_max_ymin.shape),
                                   all_pairs_min_ymax - all_pairs_max_ymin)
    
    # 5. Find min max x-coordinate for each pair
    all_pairs_min_xmax = np.minimum(xmaxA, np.transpose(xmaxB))
    all_pairs_max_xmin = np.maximum(xminA, np.transpose(xminB))
    
    # 6. Computer intersection in width
    intersect_widths = np.maximum(np.zeros(all_pairs_max_xmin.shape), 
                                  all_pairs_min_xmax - all_pairs_max_xmin)
    
    # 7. Computer intersection and union of areas
    intersection = intersect_heights * intersect_widths
    union = np.expand_dims(areaA, axis=1) + \
                            np.expand_dims(areaB, axis=0) - intersection
    
    # 8. Computer IOU(pairwise)
    IOU = (intersection / union).astype(np.float32)
    
    return IOU

In [4]:
boxesA = np.array([[142,208,158,346],
          [39, 63, 203, 112],
          [49, 75, 203, 125], 
          [31, 69, 201, 125], 
          [50, 72, 197, 121], 
          [35, 51, 196, 110]
          ]).astype(np.float32) 
          
boxesB = np.array([[243,203,348,279],
          [54, 66, 198, 114],
          [42, 78, 186, 126],
          [18, 63, 235, 135],
          [54, 72, 198, 120],
          [36, 60, 180, 108]
          ]).astype(np.float32)

In [5]:
compute_ious(boxesA, boxesB)

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.79577124, 0.48706725, 0.51433694, 0.62690467,
        0.7375334 ],
       [0.        , 0.6242775 , 0.787838  , 0.49283153, 0.79685193,
        0.42015746],
       [0.        , 0.65112543, 0.70033115, 0.609319  , 0.72605044,
        0.5192308 ],
       [0.        , 0.7406585 , 0.70739084, 0.4610215 , 0.94662803,
        0.49602544],
       [0.        , 0.6147791 , 0.39040923, 0.43102074, 0.48987743,
        0.72765553]], dtype=float32)